In [11]:
import os
import shutil
import sys
import torch
from torch.utils.data import DataLoader
import nbimporter
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score
import pandas as pd
import datetime
import matplotlib.pyplot as plt
sys.path.append("/Users/xr4good/Desktop/Ingrid/DIFF/model/")
sys.path.append("/Users/xr4good/Desktop/Ingrid/DIFF/utils/")
from model import ConTrans
from utils_3classes import get_train_dataset, get_val_dataset, get_test_dataset  ## alterar para o numero de classes
import torchvision.transforms as transforms 

BATCH_SIZE = 32
EPOCHS = 1000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)
if DEVICE.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("Atenção: executando em CPU")

# Transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Datasets
train_dataset, mean, std = get_train_dataset(transform=train_transform)
val_dataset = get_val_dataset(mean, std, transform=val_test_transform)
test_dataset = get_test_dataset(mean, std, transform=val_test_transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

model = ConTrans(attr_dim=120).to(DEVICE)
print(model)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)


best_val_acc = 0
patience = 100
counter = 0

train_losses = []
val_accuracies = []

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
##Ajustar aqui para dizer se são 2, 3 ou 6 classes
model_path = f"best_model_3classes_{timestamp}.pth"

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for imgs, attrs, labels, image_names, cell_ids in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        imgs, attrs, labels = imgs.to(DEVICE), attrs.to(DEVICE), labels.to(DEVICE)
        #print("imgs:", imgs.shape, "attrs:", attrs.shape, "labels:", labels.shape)
        outputs = model(imgs.to(DEVICE), attrs.to(DEVICE))
        #print("outputs:", outputs.shape)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_acc = 100 * correct / total
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)


    print(f"Epoch {epoch+1}: Loss {avg_train_loss:.4f}, Train Acc {train_acc:.2f}%")

    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for imgs, attrs, labels, image_names, cell_ids in tqdm(val_loader, desc="Validating"):
            imgs, attrs, labels = imgs.to(DEVICE), attrs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs, attrs)
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_acc = 100 * val_correct / val_total
    val_accuracies.append(val_acc)
    print(f"Validation Accuracy: {val_acc:.2f}%")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), model_path)
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break

# Plot training curve
plt.plot(train_losses, label="Train Loss")
plt.plot(val_accuracies, label="Val Accuracy")
plt.xlabel("Epoch")
plt.legend()
plt.title("Training Curve")
##Ajustar aqui para dizer se são 2, 3 ou 6 classes
plt.savefig(f"training_curve_3classes_{timestamp}.png")
plt.close()

# Teste
model.load_state_dict(torch.load(model_path))
model.eval()
all_preds = []
all_labels = []
all_image_names = []
all_cell_ids = []

with torch.no_grad():
    for imgs, attrs, labels, image_names, cell_ids in test_loader:
        imgs, attrs = imgs.to(DEVICE), attrs.to(DEVICE)
        outputs = model(imgs, attrs)
        _, predicted = outputs.max(1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())
        # Corrige qualquer tensor(123) para int ou str
        all_image_names.extend([imn.item() if isinstance(imn, torch.Tensor) else imn for imn in image_names])
        all_cell_ids.extend([cid.item() if isinstance(cid, torch.Tensor) else cid for cid in cell_ids])



# Salva os resultados detalhados
results_df = pd.DataFrame({
    'cell_id': all_cell_ids,
    'image_name': all_image_names,
    'true_label': all_labels,
    'predicted': all_preds
})


# Define prefixo e diretório
prefix = "test_results_3classes_experimento"
ext = ".csv"
counter = 1

# Garante nome único
while os.path.exists(f"{prefix}{counter}{ext}"):
    counter += 1

output_csv_path = f"{prefix}{counter}{ext}"
results_df.to_csv(output_csv_path, index=False)

test_acc = 100 * np.mean(np.array(all_preds) == np.array(all_labels))
print(f"Test Accuracy: {test_acc:.2f}%")
print("Confusion Matrix:\n", confusion_matrix(all_labels, all_preds))
print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=["NEGATIVE", "POSITIVE", "LIMITROFE"]))
print(f"F1 Score (macro): {f1_score(all_labels, all_preds, average='macro'):.4f}")
print(f"F1 Score (weighted): {f1_score(all_labels, all_preds, average='weighted'):.4f}")
print(f"F1 Score (micro): {f1_score(all_labels, all_preds, average='micro'):.4f}")
print(f"Precision (macro): {precision_score(all_labels, all_preds, average='macro'):.4f}")
print(f"Precision (micro): {precision_score(all_labels, all_preds, average='micro'):.4f}")
print(f"Precision (weighted): {precision_score(all_labels, all_preds, average='weighted'):.4f}")
print(f"Recall (macro): {recall_score(all_labels, all_preds, average='macro'):.4f}")
print(f"Recall (micro): {recall_score(all_labels, all_preds, average='micro'):.4f}")
print(f"Recall (weighted): {recall_score(all_labels, all_preds, average='weighted'):.4f}")

# === LEITURA DO CSV ===
csv_path = os.path.join("C:/Users/xr4good/Desktop/Ingrid/DIFF", output_csv_path)
results_df = pd.read_csv(csv_path)

# Diretórios de destino
fp_dir = "E:/datasets/imagens/erradas/3classes/falso-positivo"
fn_dir = "E:/datasets/imagens/erradas/3classes/falso-negativo"
os.makedirs(fp_dir, exist_ok=True)
os.makedirs(fn_dir, exist_ok=True)

# Diretórios onde estão as imagens 70x70 de teste
origem_dir_pos = "E:/datasets/imagens/teste/teste/3classes/teste-dir-positivo-rgb"
origem_dir_neg = "E:/datasets/imagens/teste/teste/3classes/teste-dir-negativo-rgb"
origem_dir_lim = "E:/datasets/imagens/teste/teste/3classes/teste-dir-limitrofe-rgb"

# Loop pelos resultados
for _, row in results_df.iterrows():
    image_name = row['image_name']
    true_label = row['true_label']
    predicted = row['predicted']

    if true_label != predicted:
        if (true_label == 1 or true_label == 2) and predicted == 0:
            destino_dir = fn_dir  # falso negativo
        elif true_label == 0 and (predicted == 1 or predicted == 2):
            destino_dir = fp_dir  # falso positivo
        else:
            continue

        # Verifica nos diretórios de origem onde a imagem está
        origem_img_pos = os.path.join(origem_dir_pos, image_name)
        origem_img_neg = os.path.join(origem_dir_neg, image_name)
        origem_img_lim = os.path.join(origem_dir_lim, image_name)

        if os.path.exists(origem_img_pos):
            shutil.copy2(origem_img_pos, os.path.join(destino_dir, image_name))
        elif os.path.exists(origem_img_neg):
            shutil.copy2(origem_img_neg, os.path.join(destino_dir, image_name))
        elif os.path.exists(origem_img_lim):
            shutil.copy2(origem_img_lim, os.path.join(destino_dir, image_name))
        else:
            print(f"[AVISO] Imagem não encontrada em nenhum diretório: {image_name}")

Using device: cuda
GPU: NVIDIA GeForce RTX 2080
ConTrans(
  (stem): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Dropout2d(p=0.1, inplace=False)
  )
  (patch_embedding): Conv2d(64, 64, kernel_size=(5, 5), stride=(5, 5))
  (transformer): ModuleList(
    (0-3): 4 x ModuleList(
      (0): PreNorm(
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (fn): SelfAttention(
          (attend): Softmax(dim=-1)
          (to_qkv): Linear(in_features=64, out_features=192, bias=False)
          (to_out): Sequential(
            (0): Linear(in_features=64, out_features=64, bias=True)
            (1): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (1): PreNorm(
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (fn): FeedForward(
          (net): Sequential(
        

Epoch 1: 100%|███████████████████████████████████████████████████████████████████████| 243/243 [00:23<00:00, 10.27it/s]


Epoch 1: Loss 0.6141, Train Acc 74.83%


Validating: 100%|██████████████████████████████████████████████████████████████████████| 53/53 [00:01<00:00, 30.74it/s]


Validation Accuracy: 78.32%


...

Validation Accuracy: 83.18%


Epoch 143: 100%|█████████████████████████████████████████████████████████████████████| 243/243 [00:24<00:00, 10.06it/s]


Epoch 143: Loss 0.0737, Train Acc 97.35%


Validating: 100%|██████████████████████████████████████████████████████████████████████| 53/53 [00:01<00:00, 29.76it/s]
C:\Users\xr4good\AppData\Local\Temp\ipykernel_164\2118045909.py:136: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on 

Validation Accuracy: 84.56%
Early stopping triggered.
Test Accuracy: 85.74%
Confusion Matrix:
 [[915  22  28]
 [ 51 384  48]
 [ 37  53 138]]
Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.91      0.95      0.93       965
    POSITIVE       0.84      0.80      0.82       483
   LIMITROFE       0.64      0.61      0.62       228

    accuracy                           0.86      1676
   macro avg       0.80      0.78      0.79      1676
weighted avg       0.85      0.86      0.86      1676

F1 Score (macro): 0.7899
F1 Score (weighted): 0.8553
F1 Score (micro): 0.8574
Precision (macro): 0.7979
Precision (micro): 0.8574
Precision (weighted): 0.8541
Recall (macro): 0.7828
Recall (micro): 0.8574
Recall (weighted): 0.8574
